# Recon2.2plusthreeIndependent.json

##Changing medium

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [2]:
M = cobra.io.load_json_model('Recon2.2plusthreeIndependent.json')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [3]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeIndependentMap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

model=M.copy()

In [4]:
model1=model.copy()

In [5]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model1.problem.Objective(model1.reactions.biomass_reaction_INB.flux_expression+
                                              model1.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model1.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model1.objective = quadratic_objective

In [6]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model1.reactions.EX_phe_L_e.lower_bound=-0.2
model1.reactions.EX_phe_L_e.upper_bound=-0.2
model1.reactions.EX_tyr_L_e.lower_bound=-0.04

In [7]:
## Other reactions to produce l-dopa is forbidded
model1.reactions.TYRDOPO_INB.upper_bound=0
model1.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model1.reactions.biomass_reaction.lower_bound=0.05
model1.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model1.reactions.biomass_reaction_INB.upper_bound=0.1
# Phe secretion in the blood is blocked, otherwise there is no competition.
model1.reactions.EX_phe_L_BD.upper_bound=0
# Phenylpyruvate production in the brain and Liver are limited
model1.reactions.get_by_id('PHPYRte_INB').lower_bound=0
model1.reactions.get_by_id('PHPYRte').lower_bound=0
### No PAH in the brain
model1.reactions.PHETHPTOX2_INB.lower_bound=0
model1.reactions.PHETHPTOX2_INB.upper_bound=0
model1.reactions.r0399_INB.lower_bound=0
model1.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
# model1.reactions.EX_TH_B_INB.upper_bound=1000
# model1.reactions.Phe_TH_INB.upper_bound=0
### No Tyr hydroxylase competition
model1.reactions.Phe_TH_INB.upper_bound=0
model1.reactions.PhePde_INB.upper_bound=0
model1.reactions.get_by_id('34dhpheTR_TOBD').upper_bound=0 ##No levodopa from liver to blood.
model1.solver='glpk'
sol = model1.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.3751331

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [8]:
model2=model1.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model2.genes.get_by_id('HGNC:8582').knock_out()
# Phenylpyruvate production in the brain and Liver are limited, also phe-activated.
model2.reactions.get_by_id('PHPYRte_INB').lower_bound=-0.1
model2.reactions.get_by_id('PHPYRte').lower_bound=-0.1
sol = model2.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.2140536

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

## Only Phe deprivation

In [9]:
model5=model2.copy()
model5.reactions.EX_phe_L_e.upper_bound=-0.05
model5.reactions.EX_phe_L_e.lower_bound=-0.05
sol = model5.optimize()
sol.objective_value
fbasol = model5.optimize()
b = show_map(fbasol,map_loc)
b

0.21405359999999996

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

## Only supplying Tyr

In [10]:
model3=model2.copy()
model3.reactions.EX_tyr_L_e.lower_bound=-0.2
sol = model3.optimize()
sol.objective_value
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

0.3740536

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

##Even we consider the Phe toxicity for tyrosine hydroxylase, the system uptake more tyrosine could also improve the symptoms of PKU. So, the independent pathway for Phe, Tyr and Trp taking to brain could not describe the PKU disease. 

## Combining the Phe deprivation and Tyr addition

In [11]:
model4=model3.copy()
model4.reactions.EX_phe_L_e.upper_bound=-0.05
model4.reactions.EX_phe_L_e.lower_bound=-0.05
sol = model4.optimize()
sol.objective_value
fbasol = model4.optimize()
b = show_map(fbasol,map_loc)
b

0.3740536

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

# For Phe toxicity mechanism

In [12]:
### Tyrosine hydroxylase competition
model1.reactions.TH_AToB_INB.upper_bound=0.07
model1.reactions.PhePde_INB.upper_bound=0.02
model6=model1.copy()
sol = model6.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.26800409999999997

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [17]:
model7=model6.copy()
model7.genes.get_by_id('HGNC:8582').knock_out()
# Phenylpyruvate production in the brain and Liver are activated by high Phe concentration
model7.reactions.get_by_id('PHPYRte_INB').lower_bound=-0.1
model7.reactions.get_by_id('PHPYRte').lower_bound=-0.1
sol = model7.optimize()
fluxP=sol.fluxes.loc['PheTR_TOB']
print(fluxP)
if fluxP>=0.08:
    model7.reactions.Phe_TH_INB.upper_bound=1000
    model7.reactions.Phe_TH_INB.lower_bound=0.055
sol = model7.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.08702650000000003


0.21300410000000003

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

## Only Phe deprivation

In [19]:
model8=model6.copy()
model8.genes.get_by_id('HGNC:8582').knock_out()
# Phenylpyruvate production in the brain and Liver are activated by high Phe concentration
model8.reactions.get_by_id('PHPYRte_INB').lower_bound=-0.1
model8.reactions.get_by_id('PHPYRte').lower_bound=-0.1
model8.reactions.EX_phe_L_e.upper_bound=-0.05
model8.reactions.EX_phe_L_e.lower_bound=-0.05
sol = model8.optimize()
fluxP=sol.fluxes.loc['PheTR_TOB']
print(fluxP)
if fluxP>=0.08:
    model8.reactions.Phe_TH_INB.upper_bound=1000
    model8.reactions.Phe_TH_INB.lower_bound=0.055
sol = model8.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.02594699999999997


0.21405359999999996

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

## Only supplying Tyr

In [20]:
model9=model6.copy()
model9.genes.get_by_id('HGNC:8582').knock_out()
# Phenylpyruvate production in the brain and Liver are activated by high Phe concentration
model9.reactions.get_by_id('PHPYRte_INB').lower_bound=-0.1
model9.reactions.get_by_id('PHPYRte').lower_bound=-0.1
model9.reactions.EX_tyr_L_e.lower_bound=-0.2
sol = model9.optimize()
fluxP=sol.fluxes.loc['PheTR_TOB']
print(fluxP)
if fluxP>=0.08:
    model9.reactions.Phe_TH_INB.upper_bound=1000
    model9.reactions.Phe_TH_INB.lower_bound=0.04
sol = model9.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.09546780000000213


0.22800410000000002

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

## Combining Phe deprivation and Tyr addition

In [21]:
model10=model6.copy()
model10.genes.get_by_id('HGNC:8582').knock_out()
# Phenylpyruvate production in the brain and Liver are activated by high Phe concentration
model10.reactions.get_by_id('PHPYRte_INB').lower_bound=-0.1
model10.reactions.get_by_id('PHPYRte').lower_bound=-0.1
model10.reactions.EX_phe_L_e.upper_bound=-0.05
model10.reactions.EX_phe_L_e.lower_bound=-0.05
model10.reactions.EX_tyr_L_e.lower_bound=-0.2
sol = model10.optimize()
fluxP=sol.fluxes.loc['PheTR_TOB']
print(fluxP)
if fluxP>=0.08:
    model10.reactions.Phe_TH_INB.upper_bound=1000
    model10.reactions.Phe_TH_INB.lower_bound=0.055
sol = model10.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.025946999999999974


0.26800409999999997

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…